## Public Demo of IncQuery Server Jupyter Client Extensions

### Setup IQS Connection

In [ ]:
import iqs_jupyter
iqs = iqs_jupyter.connect(
    # if you have non-guest privileges, uncomment these lines to specify your credentials 
    #address='https://openmbee.incquery.io/api',
    #user='guest',
    #password='incqueryserverguest'
)

### Select MMS commit to consider

In [ ]:
commit_selector = iqs.jupyter_tools.mms_commit_selector_widget()

In [ ]:
model = commit_selector.value().to_model_compartment()
# check if the model is indexed and loaded in memory, in which case we may proceed 
# (otherwise has to be loaded first, which is restricted to privileged users, see below)
model.is_loaded_in_memory(iqs)

### Perform custom validation checks (progress with indexed models only)

In [ ]:
iqs.validation.validate_model_compartment(model)

### Execute pre-registered query and process results

In [ ]:
iqs.queries.list_queries()

In [ ]:
qResults = iqs.query_execution.execute_query_on_model_compartment({
  "modelCompartment": model,
  "queryFQN": "iqs4mms.queries.example.dependencies"
})

In [ ]:
qResults

In [ ]:
# this is how to process the results in Python:
qResults.to_list_of_matches()

### Extract individual model elements, execute queries with parameter bindings

In [ ]:
# Descriptors of individual model elements can be extracted from query results...
first_result_element = qResults.to_list_of_matches()[0]['source']
first_result_element # check element hover for element descriptor details

In [ ]:
# ...alternatively, model element descriptors can be directly constructed using element identifiers:
some_element = model.get_element_in_compartment_by_id("_19_0_1_8760276_1525256443785_858460_4599") 
some_element

In [ ]:
# Model elements can be used to restrict the requested results
from iqs_jupyter import binding
qResults_restricted = iqs.query_execution.execute_query_on_model_compartment({
  "modelCompartment": model,
  "queryFQN": "iqs4mms.queries.example.dependencies",
  "parameterBinding": binding(target=some_element)
})
qResults_restricted

### Convert query results to Python/Jupyter-friendly Pandas dataframes and process them as such

In [ ]:
# results are now in Pandas dataframe format, ready for complex client-side manipulation
qResults_df = qResults.to_data_frame()
qResults_df

In [ ]:
# setting up Cufflinks for Pandas/Plot.ly visualization
import cufflinks as cf
cf.go_offline()

In [ ]:
# Preprocess results using Pandas and visualize the output using Plot.ly
qResults_df.applymap(lambda element: element.relative_element_id).source.value_counts().iplot(
    kind='bar', filename='cufflinks/categorical-bar-chart/frequent_sources',
    yTitle='Number of Dependencies', title='Dependencies from Source',
)

### Sandbox

## Extra section for privileged users
Do not forget to specify your privileged credentials at the top of the notebook, in the first code cell

### Repository management, indexing commits

In [ ]:
# force the server to refresh its knowledge of commits in the repository
iqs.mms_repository.update_mms_repository()

In [ ]:
iqs.persistent_index.index_model_compartment(model)

In [ ]:
iqs.in_memory_index.load_model_compartment(model)

### Register custom ad-hoc queries

In [ ]:
if "iqs4mms.queries.example.dependencies" not in iqs.queries.list_queries().query_fq_ns:
    # attempt to register query only if not already registered
    iqs.queries.register_queries_plain_text('''
pattern dependencies(source : NamedElement, target : NamedElement) {
  Dependency.supplier(dependency, target); Dependency.client(dependency, source);
}
''', query_package="iqs4mms.queries.example")

### Sandbox